In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:90%;}
div.CodeMirror {font-family:Consolas; font-size:18pt;}
div.output {font-size:19pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:18px;}
</style>
"""))

<b><font color="red" size="6">ch12. 데이터 프레임과 시리즈(Pandas)</font></b>

# 9절. 데이터 그룹화 및 집계

In [9]:
import pandas as pd
import numpy as np
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
print(iris.shape)
iris.iloc[[0,49,50,99,100,149]]

(150, 5)


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
99,5.7,2.8,4.1,1.3,versicolor
100,6.3,3.3,6.0,2.5,virginica
149,5.9,3.0,5.1,1.8,virginica


## 9.1 Group by
cf. pivot_table

In [16]:
# 1) 그룹화 방법1
#    df.groupby(by=[그룹핑할열])[함수적용해서출력할열들].기초통계함수()
    # [함수적용해서출력할열들] : 모든 열이 적용됨
iris_grouped = iris.groupby(iris.Species)
iris_grouped_sum = iris_grouped.sum()
iris_grouped_sum

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,250.3,171.4,73.1,12.3
versicolor,296.8,138.5,213.0,66.3
virginica,329.4,148.7,277.6,101.3


In [18]:
iris_grouped_sum.loc[['setosa','virginica']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,250.3,171.4,73.1,12.3
virginica,329.4,148.7,277.6,101.3


In [12]:
print(iris_grouped_sum.columns)
print(iris_grouped_sum.index)

Index(['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'], dtype='object')
Index(['setosa', 'versicolor', 'virginica'], dtype='object', name='Species')


In [19]:
iris_grouped_sum.reset_index(inplace=True) # index를 컬럼 데이터로 편입

In [21]:
iris_grouped_sum

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [25]:
# iris_grouped_sum에서 setosa 그룹함수 결과만 추출
iris_grouped_sum[iris_grouped_sum['Species']=='setosa']
iris_grouped_sum[iris_grouped_sum.Species=='setosa']
iris_grouped_sum.loc[iris_grouped_sum.Species=='setosa']

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3


In [27]:
# iris_grouped_sum에서 setosa와 virginica 그룹함수 결과만 추출
iris_grouped_sum[ (iris_grouped_sum.Species=='setosa') |
                  (iris_grouped_sum.Species=='virginica')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
2,virginica,329.4,148.7,277.6,101.3


In [29]:
iris_grouped_sum[iris_grouped_sum.Species.isin(['setosa', 'virginica'])]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
2,virginica,329.4,148.7,277.6,101.3


In [31]:
# 1) 그룹화 방법1.1 : 그룹핑할 때 index를 컬럼 데이터로 편입(reset_index 가 필요 X)
#    df.groupby(by=[그룹핑할열], as_index=False)[함수적용해서출력할열들].기초통계함수()
    # [함수적용해서출력할열들] : 모든 열이 적용됨
iris.groupby(iris.Species, as_index=False).sum()

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [35]:
display(iris.groupby(iris.Species).sum())
display(iris.groupby(['Species']).sum())
display(iris.groupby('Species').sum())

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,250.3,171.4,73.1,12.3
versicolor,296.8,138.5,213.0,66.3
virginica,329.4,148.7,277.6,101.3


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,250.3,171.4,73.1,12.3
versicolor,296.8,138.5,213.0,66.3
virginica,329.4,148.7,277.6,101.3


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,250.3,171.4,73.1,12.3
versicolor,296.8,138.5,213.0,66.3
virginica,329.4,148.7,277.6,101.3


In [38]:
# 종별 Sepal.Length와 Sepal.Width의 평균
iris.groupby('Species').mean()[['Sepal.Length','Sepal.Width']]
iris.groupby('Species')[['Sepal.Length','Sepal.Width']].mean()

,Sepal.Length,Sepal.Width
Species,,
setosa,5.006,3.428
versicolor,5.936,2.770
virginica,6.588,2.974


In [39]:
# 종별 Sepal.Length의 합계
iris.groupby('Species')['Sepal.Length'].sum() # 결과가 시리즈(1차원)

Species
setosa        250.3
versicolor    296.8
virginica     329.4
Name: Sepal.Length, dtype: float64

In [40]:
# 종별 Sepal.Length의 합계
iris.groupby('Species')[['Sepal.Length']].sum() # 결과가 데이터프레임

,Sepal.Length
Species,
setosa,250.3
versicolor,296.8
virginica,329.4


In [43]:
# 1) 그룹화 방법1
#   df.groupby(by=[그룹핑할열])[함수적용해서출력할열들].기초통계함수()
# 2) 그룹화 방법2
#   df.pivot_table(index='그룹화할 필드들',
#                  values=['함수적용해서출력할열들'],
#                  aggfunc='함수명(기본값:mean)')
iris.pivot_table(index='Species') # 종별 평균들

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,1.462,0.246,5.006,3.428
versicolor,4.260,1.326,5.936,2.770
virginica,5.552,2.026,6.588,2.974


In [44]:
#iris.groupby('Species')[['Sepal.Length','Sepal.Width']].sum()
iris.pivot_table(index='Species',
                values=['Sepal.Length','Sepal.Width'],
                aggfunc='sum')

,Sepal.Length,Sepal.Width
Species,,
setosa,250.3,171.4
versicolor,296.8,138.5
virginica,329.4,148.7


In [45]:
iris.groupby('Species').count() # 결측치를 제외한 데이터 갯수

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [47]:
iris.pivot_table(index='Species',
                aggfunc='count')# 결측치를 제외한 데이터 갯수

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [59]:
import random
random.sample(range(len(iris)), 3)

[15, 37, 39]

In [52]:
iris.shape[1]-1 # iris의 독립변수 열 갯수

4

In [60]:
# 각열에 임의의 행에 결측치를 할당
# (0번째열은 결측치1개, 1번째열은 결측치2개, 2번째열은 결측치 3개, 3번째열은 결측치4)
for col in range(iris.shape[1]-1 ):
    iris.iloc[random.sample(range(len(iris)), col+1), col] = np.nan

In [61]:
iris.groupby('Species').count()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,49,49,50,48
versicolor,50,49,50,50
virginica,50,50,47,48


In [64]:
result = iris.pivot_table(index='Species',
                aggfunc='count')
result = result[['Sepal.Length','Sepal.Width','Petal.Length','Petal.Width']]
result

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,49,49,50,48
versicolor,50,49,50,50
virginica,50,50,47,48


In [65]:
iris['Species'].value_counts() # 종별 데이터 갯수

setosa        50
versicolor    50
virginica     50
Name: Species, dtype: int64

### 다중열로 그룹화

In [67]:
iris.head(1)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,NaN,1.4,0.2,setosa


In [73]:
# iris로 다중열 그룹화하기 위해서 데이터 추가(num 필드)
# iris의 행수가 150개 = Species 3그룹 , num 6그룹
[[i]*25 for i in range(6)]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
 [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]]

In [79]:
import numpy as np
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.iloc[::10]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,NaN,1.4,0.2,setosa,0
10,5.4,3.7,1.5,0.2,setosa,0
20,5.4,3.4,1.7,0.2,setosa,0
30,4.8,3.1,1.6,0.2,setosa,1
40,5.0,3.5,1.3,0.3,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
60,5.0,2.0,3.5,1.0,versicolor,2
70,5.9,3.2,4.8,1.8,versicolor,2
80,5.5,2.4,3.8,1.1,versicolor,3
90,5.5,2.6,4.4,1.2,versicolor,3


In [81]:
# 종별, num별 합계
r = iris.groupby(by=['Species','num']).sum()
r

Sepal.Length  ...  Petal.Width
Species    num                ...             
setosa     0           120.9  ...          6.2
           1           124.6  ...          5.7
versicolor 2           150.3  ...         33.6
           3           146.5  ...         32.7
virginica  4           164.4  ...         48.7
           5           165.0  ...         48.4

[6 rows x 4 columns]

In [82]:
r.index

MultiIndex([(    'setosa', 0),
            (    'setosa', 1),
            ('versicolor', 2),
            ('versicolor', 3),
            ( 'virginica', 4),
            ( 'virginica', 5)],
           names=['Species', 'num'])

In [85]:
r.loc[['setosa','virginica']] # setosa와 virginica만 결과 추출

Sepal.Length  ...  Petal.Width
Species   num                ...             
setosa    0           120.9  ...          6.2
          1           124.6  ...          5.7
virginica 4           164.4  ...         48.7
          5           165.0  ...         48.4

[4 rows x 4 columns]

In [86]:
# 종별, num별 중위수 ('Species','num'이 컬럼 데이터로 편입)
r = iris.groupby(by=['Species','num'], as_index=False).median()
r

,Species,num,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,0,5.05,3.40,1.50,0.2
1,setosa,1,5.00,3.40,1.50,0.2
2,versicolor,2,6.10,2.80,4.50,1.4
3,versicolor,3,5.70,2.75,4.20,1.3
4,virginica,4,6.50,2.90,5.55,2.0
5,virginica,5,6.40,3.00,5.50,2.0


In [ ]:
# setosa와 virginica 추출